In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 56.4 MB/s eta 0:00:00


In [2]:
from tqdm.auto import tqdm
import pandas as pd
tqdm.pandas()

In [3]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity

In [4]:
DRIVE_PATH = '/content/drive/MyDrive/question_answering_data/'

In [5]:
# load data
wiki_df = pd.read_csv(DRIVE_PATH + 'processed/wikipedia_20220620_cleaned_v2-1.csv')
wiki_df

,title,text,bm25_text
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",trang_chính templatestyles srcwiki2021stylescs...
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...,internet society isoc tổ_chức quốc_tế hoạt_độn...
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",tiếng việt gọi tiếng việt_nam việt ngữ ngôn_ng...
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố...",hệ_thống thanh_điệu phát_triển hệ_thống thanh_...
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...,tiếp_xúc hán – việt thành 2 giai_đoạn bullet g...
...,...,...,...
1944401,VNOI,chuyên viên của các tập đoàn công nghệ lớn như...,chuyên_viên tập_đoàn công_nghệ google facebook...
1944402,Vòng hoa,"Vòng hoa (; IPA: ) là một loại hoa, lá, quả, ...",vòng_hoa ipa hoa_lá cành vật_liệu kết_cấu thàn...
1944403,Vòng hoa,vật trong thần thoại La Mã và Hy Lạp được miêu...,vật thần_thoại la mã hy lạp miêu_tả đeo vòng_h...
1944404,Vòng hoa,"một vai trò nổi bật, tương tự như ở Anh.\n\n==...",vai_trò nổi_bật tương_tự biểu_tượng vòng_hoa t...


In [13]:
import re
from nltk import word_tokenize as lib_tokenizer
import string
from pyvi import ViTokenizer
import nltk
nltk.download('punkt')

# stop_words_file = '/content/drive/MyDrive/question_answering_data/vietnamese-stopwords.txt'
stop_words_file = '/content/drive/MyDrive/question_answering_data/vietnamese-stopwords-dash.txt'
with open(stop_words_file, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

print(stopwords)


dict_map = dict({})
def word_tokenize(text):
    global dict_map
    words = text.split()
    words_norm = []
    for w in words:
        if dict_map.get(w, None) is None:
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"')
        words_norm.append(dict_map[w])
    return words_norm

def strip_context(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    filtered_text = ' '.join(filtered_words)

    return filtered_text

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()

    x = re.sub(r'[{}]+'.format(string.punctuation), '', x) # remove punctuation
    x = ViTokenizer.tokenize(x)

    x = remove_stopwords(x)
    return x

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy', 'ba', 'ba_ba', 'ba_bản', 'ba_cùng', 'ba_họ', 'ba_ngày', 'ba_ngôi', 'ba_tăng', 'bao_giờ', 'bao_lâu', 'bao_nhiêu', 'bao_nả', 'bay_biến', 'biết', 'biết_bao', 'biết_bao_nhiêu', 'biết_chắc', 'biết_chừng_nào', 'biết_mình', 'biết_mấy', 'biết_thế', 'biết_trước', 'biết_việc', 'biết_đâu', 'biết_đâu_chừng', 'biết_đâu_đấy', 'biết_được', 'buổi', 'buổi_làm', 'buổi_mới', 'buổi_ngày', 'buổi_sớm', 'bà', 'bà_ấy', 'bài', 'bài_bác', 'bài_bỏ', 'bài_cái', 'bác', 'bán', 'bán_cấp', 'bán_dạ', 'bán_thế', 'bây_bẩy', 'bây_chừ', 'bây_giờ', 'bây_nhiêu', 'bèn', 'béng', 'bên', 'bên_bị', 'bên_có', 'bên_cạnh', 'bông', 'bước', 'bước_khỏi', 'bước_tới', 'bước_đi', 'bạn', 'bản', 'bản_bộ', 'bản_riêng', 'bản_thân', 'bản_ý', 'bất_chợt', 'bất_cứ', 'bất_giác', 'bất_kì', 'bất_kể', 'bất_kỳ', 'bất_luận', 'bất_ngờ', 'bất_nhược', 'bất_quá', 'bất_quá_chỉ', 'bất_thình_lình', 'bất_tử', 'bất_đồ', 'bấy', 'bấy_chầy', 'bấy_chừ', 'bấy_giờ', 'bấy_lâu', 'bấy_lâ

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# wiki_df['bm25_text'] = wiki_df['bm25_text'].apply(lambda x: str(x).lower()).progress_apply(post_process)
corpus = [str(x).split() for x in wiki_df['bm25_text'].values]
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save(DRIVE_PATH + "model/bm25_text/dictionary.dict")
tfidf_model.save(DRIVE_PATH + "model/bm25_text/tfidf_model.model")
bm25_index.save(DRIVE_PATH + "model/bm25_text/bm25_index.index")

In [15]:
del corpus
del dictionary
del bm25_model
del bm25_corpus
del bm25_index
del tfidf_model

#BM25 Title

In [14]:
wiki_df['title_lower'] = wiki_df['title'].progress_apply(lambda x: str(x).lower()).progress_apply(post_process)
wiki_df

  0%|          | 0/1944406 [00:00<?, ?it/s]

  0%|          | 0/1944406 [00:00<?, ?it/s]

,title,text,bm25_text,title_lower
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",trang_chính templatestyles srcwiki2021stylescs...,trang
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...,internet society isoc tổ_chức quốc_tế hoạt_độn...,internet society
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",tiếng việt gọi tiếng việt_nam việt ngữ ngôn_ng...,tiếng việt
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố...",hệ_thống thanh_điệu phát_triển hệ_thống thanh_...,tiếng việt
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...,tiếp_xúc hán – việt thành 2 giai_đoạn bullet g...,tiếng việt
...,...,...,...,...
1944401,VNOI,chuyên viên của các tập đoàn công nghệ lớn như...,chuyên_viên tập_đoàn công_nghệ google facebook...,vnoi
1944402,Vòng hoa,"Vòng hoa (; IPA: ) là một loại hoa, lá, quả, ...",vòng_hoa ipa hoa_lá cành vật_liệu kết_cấu thàn...,vòng_hoa
1944403,Vòng hoa,vật trong thần thoại La Mã và Hy Lạp được miêu...,vật thần_thoại la mã hy lạp miêu_tả đeo vòng_h...,vòng_hoa
1944404,Vòng hoa,"một vai trò nổi bật, tương tự như ở Anh.\n\n==...",vai_trò nổi_bật tương_tự biểu_tượng vòng_hoa t...,vòng_hoa


In [16]:
wiki_df.to_csv("/content/drive/MyDrive/question_answering_data/processed/wikipedia_20220620_cleaned_v2-1_title.csv",index=False)

In [17]:
corpus = [doc.split() for doc in wiki_df['title_lower']] #simple tokenier

dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')

In [19]:
dictionary.save(DRIVE_PATH +"model/bm25_title/dictionary.dict")
tfidf_model.save(DRIVE_PATH +"model/bm25_title/tfidf_model.model")
bm25_index.save(DRIVE_PATH +"model/bm25_title/bm25_index.index")

#End